# **Project 3 Group 4 Problem 2 - Task 1 (Dijone M.)**

In [13]:
# Implementing Mini Block Cipher with key size 16 bit and block size 16 bit (Task 1)
# Includes: S-box, ShiftRows, Mix, AddRoundKey, Key expansion,
# encrypt_round1/2, decrypt_round2/1, encrypt, decrypt,generation of plaintext-ciphertext pairs

In [14]:
# =============================
# Cell 1 - Imports and Utilities
# =============================
import random

def split_words(x):
    "Splitting a 16-bit size word into 4 nibbles"

    return [ (x >> 12) & 0xF, (x >> 8) & 0xF, (x >> 4) & 0xF, x & 0xF ]

def join_nibbles(nibs):
    "Joining 4 nibbles into a 16-bit integer"
    return ((nibs[0] & 0xF) << 12) | ((nibs[1] & 0xF) << 8) | ((nibs[2] & 0xF) << 4) | (nibs[3] & 0xF)

In [15]:
# =============================
# Cell 2 - S-box and inverse S-box
# =============================

# 4-bit S-box (SAES)
SBOX = [0x9,0x4,0xA,0xB,0xD,0x1,0x8,0x5,0x6,0x2,0x0,0x3,0xC,0xE,0xF,0x7]
INV_SBOX = [0]*16
for i,v in enumerate(SBOX):
    INV_SBOX[v] = i

def substitute(state):
    "Applying S-box to each nibble"
    return [SBOX[x & 0xF] for x in state]

def inv_substitute(state):
    "Applying the inverse of S-box to each nibble"
    return [INV_SBOX[x & 0xF] for x in state]


In [16]:
# =============================
# Cell 3 - ShiftRows and inverse ShiftRows
# =============================

def shiftrows(state):
    "Swapping s2 and s3 in order to do the shiftrow"
    return [state[0], state[1], state[3], state[2]]

def inv_shiftrows(state):
    "Applying the inverse of the shiftrows"
    return [state[0], state[1], state[3], state[2]]

In [17]:
# =============================
# Cell 4 - MixColumns, Inverse MixColumns, and GF(16) Multiplication
# =============================

# GF(16) multiplication with modulo x^4 + x + 1 for SAES 
def gf_multiplication(a, b):
    p = 0
    for i in range(4):               
        if (b >> i) & 1:
            p ^= (a << i)
    for i in range(7, 3, -1):        #  reduce modulo 0x13
        if (p >> i) & 1:
            p ^= 0x13 << (i - 4)
    return p & 0xF

# SAES Mix + InvMix (2x2) 
def mix(state):
    "SAES MixColumns - matrix [[1,4],[4,1]] over GF(16)"
    s0, s1, s2, s3 = [x & 0xF for x in state]
    temp0 = (s0 ^ gf_multiplication(4, s2)) & 0xF
    temp2 = (gf_multiplication(4, s0) ^ s2) & 0xF
    temp1 = (s1 ^ gf_multiplication(4, s3)) & 0xF
    temp3 = (gf_multiplication(4, s1) ^ s3) & 0xF
    return [temp0, temp1, temp2, temp3]

def inv_mix(state):
    "Inverse MixColumns - matrix [[1,4],[4,1]] over GF(16)"
    s0, s1, s2, s3 = [x & 0xF for x in state]
    temp0 = (gf_multiplication(9, s0) ^ gf_multiplication(2, s2)) & 0xF
    temp2 = (gf_multiplication(2, s0) ^ gf_multiplication(9, s2)) & 0xF
    temp1 = (gf_multiplication(9, s1) ^ gf_multiplication(2, s3)) & 0xF
    temp3 = (gf_multiplication(2, s1) ^ gf_multiplication(9, s3)) & 0xF
    return [temp0, temp1, temp2, temp3]

In [20]:
# =============================
# Cell 5 - AddRoundKey
# =============================

def add_round_key(state, round_key):
    "XORing the 4 nibbles with the round_key"
    key_nibs = split_words(round_key)
    return [(state[i] ^ key_nibs[i]) & 0xF for i in range(4)]

In [21]:
# =============================
# Cell 6 - Key expansion
# =============================

def key_expansion(K):
    # Splitting 16-bit key into two 8-bit words
    word0 = (K >> 8) & 0xFF
    word1 = K & 0xFF

    # Making sure to round constants
    RCONST1 = 0x80  # 1000 0000
    RCONST2 = 0x30  # 0011 0000

    # Swapping nibbles in an 8-bit word
    def Rotate_Nibbles(b):  
        return ((b << 4) | (b >> 4)) & 0xFF

    # Applying 4-bit SBOX to each of the nibbles
    def Sub_Nibbles(b):  
        high_nibble = SBOX[(b >> 4) & 0xF]
        low_nibble = SBOX[b & 0xF]
        return ((high_nibble << 4) | low_nibble) & 0xFF

    # Using the SAES schedule
    word2 = word0 ^ RCONST1 ^ Sub_Nibbles(Rotate_Nibbles(word1))
    word3 = word2 ^ word1
    word4 = word2 ^ RCONST2 ^ Sub_Nibbles(Rotate_Nibbles(word3))
    word5 = word3 ^ word4

    # Rounding keys for the two round ciphers
    K1 = (word2 << 8) | word3
    K2 = (word4 << 8) | word5
    return K1, K2


In [22]:
# =============================
# Cell 7 - Round functions
# =============================

# Two rounds of encryption to get the ciphertext.
def encrypt_round1(state_bits, K1):
    "encrypt_round1() using K1 -> i. Subsititute(); ii. Shift(); iii. Mix(); iv. AddRoundKey()"
    s = split_words(state_bits)
    s = substitute(s)
    s = shiftrows(s)
    s = mix(s)
    s = add_round_key(s, K1)
    return join_nibbles(s)

def encrypt_round2(state_bits, K2):
    "encrypt_round2() using K2 -> i. Subsititute(); ii. Shift() ; iii. AddRoundKey()"
    s = split_words(state_bits)
    s = substitute(s)
    s = shiftrows(s)
    s = add_round_key(s, K2)
    return join_nibbles(s)

# Two rounds of decryption to recover the plaintext
def decrypt_round2(state_bits, K2):
    "decrypt_round2() using K2 -> i. AddRoundKey(); ii. Shift(); iii. Subsititute()"
    s = split_words(state_bits)
    s = add_round_key(s, K2)
    s = inv_shiftrows(s)
    s = inv_substitute(s)
    return join_nibbles(s)

def decrypt_round1(state_bits, K1):
    "decrypt_round1() using K1 -> i. AddRoundKey(); ii. Mix(); iii. Shift(); iv. Subsititute()"
    s = split_words(state_bits)
    s = add_round_key(s, K1)
    s = inv_mix(s)
    s = inv_shiftrows(s)
    s = inv_substitute(s)
    return join_nibbles(s)



In [23]:
# =============================
# Cell 8 - Full encryption and decryption 
# =============================

def encrypt(plaintext, K):
    "Encrypting a 16-bit plaintext with 16-bit key in order to get a 16-bit ciphertext"
    K1, K2 = key_expansion(K)
    intermediate_ciphertext = encrypt_round1(plaintext, K1)
    ciphertext = encrypt_round2(intermediate_ciphertext, K2)
    return ciphertext

def decrypt(ciphertext, K):
    "Decrypting a 16-bit ciphertext with key in order to get the 16-bit plaintext"
    K1, K2 = key_expansion(K)
    intermediate_plaintext = decrypt_round2(ciphertext, K2)
    plaintext = decrypt_round1(intermediate_plaintext, K1)
    return plaintext



In [24]:
# =============================
# Cell 9 - Create at least 10 plaintext-ciphertext pairs
# =============================

def create_pairs(n=12, seed=None):
    "Creation of at least ten distinct plaintext–ciphertext pairs using random 16-bit keys"
    if seed is not None:
        random.seed(seed)

    pairs = []

    for _ in range(n):
        K = random.randint(0, 0xFFFF)
        P = random.randint(0, 0xFFFF)
        C = encrypt(P, K)
        K1, K2 = key_expansion(K)
        pairs.append({
            "key_hex": f"{K:04X}",
            "K1_hex": f"{K1:04X}",
            "K2_hex": f"{K2:04X}",
            "plaintext_hex": f"{P:04X}",
            "ciphertext_hex": f"{C:04X}"
        })
    return pairs

In [25]:
# =============================
# Cell 10 -  Display of the created more than 10 plaintext-ciphertext pairs
# =============================

# Creating 12 distinct plaintext–ciphertext pairs and displaying them in hex format
pairs = create_pairs(12, seed=12345) #seed in order to keep repeating
for row in pairs:
    print(row)


{'key_hex': 'D54F', 'K1_hex': '2867', 'K2_hex': '4027', 'plaintext_hex': '0534', 'ciphertext_hex': '4B45'}
{'key_hex': '98E7', 'K1_hex': '47A0', 'K2_hex': 'E747', 'plaintext_hex': 'BCA0', 'ciphertext_hex': 'F450'}
{'key_hex': '6327', 'K1_hex': 'B99E', 'K2_hex': '7BE5', 'plaintext_hex': '8A5D', 'ciphertext_hex': 'E599'}
{'key_hex': 'DF59', 'K1_hex': '7E27', 'K2_hex': '1433', 'plaintext_hex': '52C8', 'ciphertext_hex': 'F02A'}
{'key_hex': 'BF02', 'K1_hex': '9694', 'K2_hex': '74E0', 'plaintext_hex': '3F9F', 'ciphertext_hex': '96CD'}
{'key_hex': 'DDA9', 'K1_hex': '7DD4', 'K2_hex': '9347', 'plaintext_hex': '85AD', 'ciphertext_hex': '5FD1'}
{'key_hex': '5943', 'K1_hex': '6427', 'K2_hex': '0E29', 'plaintext_hex': '5FA5', 'ciphertext_hex': 'B54C'}
{'key_hex': 'B5B5', 'K1_hex': '2693', 'K2_hex': 'A437', 'plaintext_hex': '2EBE', 'ciphertext_hex': 'E511'}
{'key_hex': 'D301', 'K1_hex': '1A1B', 'K2_hex': '1E05', 'plaintext_hex': '5549', 'ciphertext_hex': '7EA0'}
{'key_hex': '4BD9', 'K1_hex': 'E53C',